# Binlog from Kafka to MySQL (Consumer)

### importing necessary libraries

In [1]:
import json
from confluent_kafka import Consumer
import pymysql
from pymysql import err

### following function mysql_database() is responsible for replicating the data from another mysql server to the desire one

In [2]:
# change as per your requirement
topic_name=['TestDatabase.TestTable']  # Kafka topic(s) to subscribe to

def mysql_database(message_data):
    """
    Function to handle MySQL database operations (INSERT, UPDATE, DELETE, QUERY)
    based on the data received from the Kafka topic.
    
    Parameters:
        message_data (dict): A dictionary containing the database, table, and operation type 
                             (INSERT, UPDATE, DELETE, QUERY), and the respective data.
    """
    connection = None
    cursor = None

    try:
        # Connect to MySQL database
        connection = pymysql.connect(
            host='127.0.0.1',  # Localhost server
            user='root',  # MySQL root user
            password='Mysql@123',  # MySQL root password
            port=3307  # MySQL port
        )
        
        cursor = connection.cursor()

        # Ensure the specified database exists, create if not
        database = message_data['database']
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{database}`")
        connection.select_db(database)

        table = message_data['table']  # Get the table name from the message



        # Perform the respective operation based on the message type
        if message_data['type'] == 'INSERT':
            # Prepare an INSERT statement with dynamic columns and values
            data = message_data['data']
            columns = ', '.join(data.keys())
            column_values = ', '.join(['%s'] * len(data))

            insert_query = f"INSERT INTO `{table}` ({columns}) VALUES ({column_values})"
            print(f"Executing query: {insert_query}")

            # Execute the insert query
            cursor.execute(insert_query, tuple(data.values()))
            connection.commit()
            print(f"successfully executed query: INSERT INTO `{table}` ({columns}) VALUES ({column_values})")

        elif message_data['type'] == 'UPDATE':
            # Prepare an UPDATE statement with set and where clauses
            before_data = message_data['before']
            after_data = message_data['after']

            set_clause = ', '.join(f"`{k}` = %s" for k in after_data.keys())
            where_clause = ' AND '.join(f"`{k}` = %s" for k in before_data.keys())

            update_query = f"UPDATE `{table}` SET {set_clause} WHERE {where_clause}"
            print(f"Executing query: {update_query}")

            # Execute the update query
            cursor.execute(update_query, tuple(after_data.values()) + tuple(before_data.values()))
            connection.commit()
            print(f"successfully executed query: UPDATE `{table}` SET {set_clause} WHERE {where_clause}")

        elif message_data['type'] == 'DELETE':
            # Prepare a DELETE statement with a where clause
            delete_data = message_data['data']
            where_clause = ' AND '.join(f"`{k}` = %s" for k in delete_data.keys())

            delete_query = f"DELETE FROM `{table}` WHERE {where_clause}"
            print(f"Executing query: {delete_query}")

            # Execute the delete query
            cursor.execute(delete_query, tuple(delete_data.values()))
            connection.commit()
            print(f"successfully executed query: DELETE FROM `{table}` WHERE {where_clause}")

        elif message_data['type'] == 'QUERY':
            # Execute any general query passed via the 'query' key
            query = message_data['query']
            print(f"Executing query: {query}")
            cursor.execute(query)
            connection.commit()
            print(f"successfully executed query: {query}")

    except (pymysql.MySQLError, err.OperationalError) as e:
        print(f"Database error: {e}")
        if connection is not None:
            connection.rollback()  # Roll back any uncommitted changes on error

    finally:
        # Ensure cursor and connection are closed after execution
        if cursor is not None:
            cursor.close()
        if connection is not None:
            connection.close()

### following function kafka_consumer() is responsible for connecting kafka server and extracting data from desire kafka topics

In [3]:

def kafka_consumer():
    """
    Function to consume messages from a Kafka topic and process the message data.
    
    This function:
    - Connects to the Kafka broker
    - Subscribes to a topic
    - Processes incoming messages by decoding JSON and passing the data to the MySQL handler
    """
    kafka_consumer_config = {
        'bootstrap.servers': 'localhost:9092',  # Kafka broker address
        'group.id': 'apple2',  # Consumer group ID
        'auto.offset.reset': 'earliest'  # Start from the earliest offset if no previous offset is found
    }

    # Initialize the Kafka consumer
    consumer = Consumer(kafka_consumer_config)

    topic = topic_name
    consumer.subscribe(topic)

    print(f"{topic} topic is subscribed.")

    try:
        # Continuously poll Kafka for new messages
        while True:
            msg = consumer.poll(1.0)  # Poll every 1 second

            if msg is None:
                continue  # No new messages, continue polling

            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue  # Log consumer error and skip to the next message

            try:
                # Decode and process the JSON message
                message_data = json.loads(msg.value().decode('utf-8'))
                mysql_database(message_data)  # Call the MySQL handler to process the message

            except json.JSONDecodeError as e:
                print(f"Error while decoding JSON: {e}")  # Log JSON decoding errors and skip the message
                continue

            except Exception as e:
                print(f"Error processing message: {e}")  # Log any other processing errors

    except Exception as e:
        print(f"Error while loading messages: {e}")  # Handle unexpected errors in Kafka polling loop

    finally:
        # Close the Kafka consumer after the loop ends
        consumer.close()


if __name__ == '__main__':
    # Main entry point for the Kafka consumer
    kafka_consumer()


['TestDatabase.TestTable'] topic is subscribed.
Executing query: create table TestTable (id int,name varchar(25))
successfully executed query: create table TestTable (id int,name varchar(25))
Executing query: insert into TestTable values(1,'susan')
successfully executed query: insert into TestTable values(1,'susan')
Executing query: insert into TestTable values(2,'abib')
successfully executed query: insert into TestTable values(2,'abib')
Executing query: delete from TestTable where id =1
successfully executed query: delete from TestTable where id =1
Executing query: update TestTable set name='anita' where id=2
successfully executed query: update TestTable set name='anita' where id=2


KeyboardInterrupt: 